# 🧭 Projet - Problème du Voyageur de Commerce (TSP)

---

Ce projet explore des heuristiques pour résoudre le **Problème du Voyageur de Commerce (Travelling Salesman Problem)**.
Nous allons implémenter et comparer différentes approches pour trouver un circuit de coût minimal visitant chaque ville une seule fois.

---

👨‍💻 Myriam HADDOUK & Myriam SCHULMANN
🎓 Projet Mathématiques-informatique
📅 Juin 2025


**Imports**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math

### **City Generation**

In [ ]:
def generate_cities(n: int, seed: int = None) -> np.ndarray:
    """
    Generate a set of cities with random (x, y) coordinates in a 1000x1000 grid.

    Parameters:
    - n (int): Number of cities to generate.
    - seed (int, optional): Random seed for reproducibility.

    Returns:
    - np.ndarray: Array of shape (n, 2) with x and y coordinates of each city.
    """
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    return np.random.randint(0, 1000, size=(n, 2))

In [ ]:
cities = generate_cities(20, seed=42)
print(cities)

**Distance Matrix**

In [ ]:
def compute_distance_matrix(cities: np.ndarray) -> np.ndarray:
    """
    Compute the pairwise Euclidean distance matrix between all cities.

    Parameters:
    - cities (np.ndarray): Array of shape (n, 2) with x and y coordinates.

    Returns:
    - np.ndarray: Array of shape (n, n) with distances between each pair of cities.
    """
    n = cities.shape[0]
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                dx = cities[i][0] - cities[j][0]
                dy = cities[i][1] - cities[j][1]
                matrix[i][j] = math.hypot(dx, dy)

    return matrix

In [ ]:
distance_matrix = compute_distance_matrix(cities)
print(distance_matrix)